In [53]:
# load the library
from bs4 import BeautifulSoup as Soup
import urllib2, requests
import pandas as pd

df = pd.DataFrame()   # create a new data frame
pd.set_option('max_colwidth',500)    # to remove column limit (Otherwise, we'll lose some info)


# indeed.com's query url
base_url = 'http://www.indeed.com/jobs?q=data+scientist&jt=fulltime&sort='
sort_by = 'date'          # sort by data
start_from = '&start='    # start page number

In [54]:
for page in range(1,11): # page from 1 to 100 
    page = (page-1) * 10  
    url = "%s%s%s%d" % (base_url, sort_by, start_from, page) # get full url 
    contents = Soup(urllib2.urlopen(url), "lxml") 

    contentsElements = contents.findAll('div', attrs={'class' : '  row  result'}) 
    # each row contains 1 job
    
    # extract job information
    for elem in contentsElements: 
        try:
            company = elem.find('span', attrs={'itemprop':'name'}).getText().strip()
        except AttributeError:
            company = '-'
        job_title = elem.find('a', attrs={'class':'turnstileLink'}).attrs['title']
        home_url = "http://www.indeed.com"
        job_link = "%s%s" % (home_url,elem.find('a').get('href'))
        job_addr = elem.find('span', attrs={'itemprop':'addressLocality'}).getText()
        job_posted = elem.find('span', attrs={'class': 'date'}).getText()


        # add a job info to our data frame
        df = df.append({'company': company, 'job_title': job_title, 
                        'job_link': job_link, 'job_posted': job_posted,
                        'job_location': job_addr
                       }, ignore_index=True)

df    

,company,job_link,job_location,job_posted,job_title
0,"Anthem, Inc.",http://www.indeed.com/rc/clk?jk=f30c6cfd9ae34599&fccid=2a4da7fa99f4b9ae,"Chicago, IL 60654 (Loop area)",Just posted,Data Scientist or Data Scientist Lead(Healthcare)-115937
1,"Renaissance Learning, Inc.",http://www.indeed.com/rc/clk?jk=a444bdb79a55f6d4&fccid=10a6caa919d4cfca,"Vancouver, WA 98662",Just posted,"Research Scientist, Learning Science"
2,Delanecs,http://www.indeed.com/cmp/Delanecs/jobs/Data-Analytic-Scientist-e7aba04afe9b793c?r=1&fccid=ee83ce511df944f7,"Alpharetta, GA",Just posted,Data Analytic Scientist (ES) - New New
3,Lawrence Berkeley National Laboratory,http://www.indeed.com/rc/clk?jk=0fdad61b9b07acf9&fccid=5bee64c47451d1c2,"Berkeley, CA",Just posted,Research Scientist (Career-Track)
4,"MacAulay-Brown, Inc.",http://www.indeed.com/rc/clk?jk=daff67291bf1b4b0&fccid=b253b564901b80b5,"Dayton, OH 45430",Just posted,Operations Research Analyst
5,Streck,http://www.indeed.com/rc/clk?jk=ca6fd423762a1e33&fccid=0133f7a60e8ad6a2,"Omaha, NE",Just posted,Research and Development Scientist
6,"Carfax, Inc.",http://www.indeed.com/rc/clk?jk=e1b3f19f512cb5b8&fccid=4f7b77bc3ccc0c98,"Centreville, VA",Just posted,Manager - Business Data Analysis
7,Google,http://www.indeed.com/rc/clk?jk=1c3afb54a4ff1688&fccid=a5b4499d9e91a5c6,"Cambridge, MA 02142 (East Cambridge area)",Just posted,"Computational Biologist, Verily Life Sciences"
8,Google,http://www.indeed.com/rc/clk?jk=6fa3f9bc45d4d198&fccid=a5b4499d9e91a5c6,"Mountain View, CA",Just posted,Candidate Offer Assessment Specialist
9,Streck,http://www.indeed.com/rc/clk?jk=507c8dc8aff8a1cb&fccid=0133f7a60e8ad6a2,"Omaha, NE",Just posted,Market Research Analyst


In [55]:
# check one entry to verify contents
print df.iloc[6]

company                                                                    Carfax, Inc.
job_link        http://www.indeed.com/rc/clk?jk=e1b3f19f512cb5b8&fccid=4f7b77bc3ccc0c98
job_location                                                            Centreville, VA
job_posted                                                                  Just posted
job_title                                              Manager - Business Data Analysis
Name: 6, dtype: object


In [56]:
# Save the result to CSV
df.to_csv('./indeed_companies.csv', encoding='utf-8')